In [2]:
from dotenv import load_dotenv

load_dotenv()

True

# agent with memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

HumanTemplate = """"""
ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="너는 뛰어나고 유능한 주문봇이야."
            ),
        HumanMessagePromptTemplate.from_template
    ]

)

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 뛰어나고 유능한 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

runnable = prompt | model
runnable

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='너는 뛰어나고 유능한 주문봇이야이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002A041656E60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002A0416A8D00>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [11]:
with_message_history.invoke(
    {"input": "안녕. 나는 '나들'이라고 해. 너는 어떤 로봇이니?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 6eff1f89-2be1-4866-a05e-b025206d1518 not found for run 0cd2d087-5827-40cf-946d-343b7da9153e. Treating as a root run.


AIMessage(content='안녕하세요, 나들님! 저는 주문봇입니다. 주문하고 싶은 상품이나 음식 등을 제가 도와드릴 수 있어요. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 194, 'total_tokens': 261}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1df4a857-fb9e-4dcd-bba3-87f4b2285f2d-0')

In [12]:
with_message_history.invoke(
    {"input": "내 이름을 기억해?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 3848fd0b-6585-4b35-b6d1-6a5f3f4c6069 not found for run 907c7b82-be7e-40ef-96c1-f2422815ea26. Treating as a root run.


AIMessage(content="네, '나들'이라는 이름을 기억했습니다. 어떤 도움이 필요하신가요, 나들님?", response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 277, 'total_tokens': 318}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-07adc98e-96d9-4d14-9328-558c48b2f5bb-0')

# tool 사용

In [ ]:
from langchain.tools import tool

@tool
def get_recent_orders_by_status(username: str, status: str) -> list:
    pass

In [ ]:
tools = [get_recent_orders_by_status]

tools만 여러 개 생성하고 나머지 model, prompt는 하나만 있어도 될 듯

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "내 등록 id는 nadle인데 지난 주문 내역에 대해 알고 싶어"
    }
)

## multiple tools

In [ ]:
@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return Order.objects.filter(
        user=user,
        status_changes__status=status
    ).distinct().order_by('-created_at')[:3]

In [16]:
from langchain_core.tools import tool

@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

In [17]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = [get_recent_orders_by_status]
llm_with_tools = model.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

In [18]:
def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

In [19]:
chain = llm_with_tools | call_tools

In [20]:
chain.invoke("내 아이디는 '나들'이라고 해. '주문 완료' 상태인 주문들을 모두 알고 싶어")

[{'name': 'get_recent_orders_by_status',
  'args': {'user': '나들', 'status': '주문 완료'},
  'id': 'call_YZC0nZ9xXQlrLCsHYtjyy5Vz',
  'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}]

# memory + tools

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 뛰어나고 유능한 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

In [6]:
from langchain_core.tools import tool

@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

tools = [get_recent_orders_by_status]

In [7]:
runnable_with_tools = prompt | model.bind_tools(tools)
runnable_with_tools

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='너는 뛰어나고 유능한 주문봇이야이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020DFFE60760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020DFFE92170>, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'fun

In [11]:
runnable_with_tools.invoke(
    {"input": "내 아이디는 '나들'이라고 해. '주문 완료' 상태인 주문들을 모두 알고 싶어",
     "chat_history": []},
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_398At3WhODfSHxUKTfVCTGWO', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 169, 'total_tokens': 195}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2cbb90ca-3065-4187-99f7-50a08ed96ce4-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'user': '나들', 'status': '주문 완료'}, 'id': 'call_398At3WhODfSHxUKTfVCTGWO'}])

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [18]:
with_message_history.invoke(
    {"input": "안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"},
    config={"configurable": {"session_id": "test2"}},
)

Parent run 6015828a-8587-4edc-9c7a-d9682067e55b not found for run 8c5c5415-c501-46fc-95e3-0da9ccd053d4. Treating as a root run.


AIMessage(content='안녕, 나들님! 저는 주문봇이에요. 주문과 관련된 정보를 제공해드릴 수 있어요. 궁금한 게 있으시면 언제든지 물어보세요. 😊\n안녕, 나들님! 주문과 관련된 정보를 제공해드릴 수 있는 주문봇이에요. 궁금한 점이 있으시면 언제든지 물어보세요. 함께 도와드릴게요! 😊', response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 164, 'total_tokens': 309}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-665d1537-d715-4122-9f44-0412807e5e83-0')

In [19]:
with_message_history.invoke(
    {"input": "내 이름이 뭐라고?"},
    config={"configurable": {"session_id": "test2"}},
)

Parent run 8ef78b05-b008-417d-994e-c433e27bc2e3 not found for run 71a51484-8f4f-4f8f-a242-d32d2667b7cb. Treating as a root run.


AIMessage(content="죄송합니다, '나들'이라고 하셨죠! 잊지 않고 기억하고 있습니다. 무엇을 도와드릴까요? 😊", response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 324, 'total_tokens': 376}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-746033fb-4164-4579-aa00-7a19c352bb6f-0')

In [20]:
with_message_history.invoke(
    {"input": "내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"},
    config={"configurable": {"session_id": "test2"}},
)

Parent run 47be1cf4-d2f2-4bfe-aeb5-820984c41b9b not found for run 828986b0-700f-4a66-b562-338650d3c831. Treating as a root run.


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_31oKhBjry7v1DyuaQDQBL5Ke', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 420, 'total_tokens': 446}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-68623cef-e9a7-46ba-beba-d5ca49908dd1-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'user': '나들', 'status': '주문 완료'}, 'id': 'call_31oKhBjry7v1DyuaQDQBL5Ke'}])

In [27]:
with_message_history.invoke(
    {"input": "내 이름이 뭐랬지?"},
    config={"configurable": {"session_id": "test2"}},
)

Parent run 1e655132-bd4f-44c7-a980-48eb362451ad not found for run 157c7f35-d080-49a7-80f8-e357e53e6312. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_31oKhBjry7v1DyuaQDQBL5Ke", 'type': 'invalid_request_error', 'param': 'messages.[7].role', 'code': None}}

In [23]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

In [24]:
chain = with_message_history | call_tools

In [25]:
chain.invoke(
    {"input": "다시 한 번 알려줘. 내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"},
    config={"configurable": {"session_id": "test1"}},
)

Parent run eec4c1f8-9010-45d2-ad1e-e0ce8c7623ea not found for run b4dbf8c3-3270-42fd-a34d-f2afd1acde9f. Treating as a root run.


[{'name': 'get_recent_orders_by_status',
  'args': {'user': '나들', 'status': '주문 완료'},
  'id': 'call_RtKlm0aQkorTIlnT8Zpgu8NL',
  'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}]

In [26]:
chain.invoke(
    {"input": "내 이름이 뭐랬지?"},
    config={"configurable": {"session_id": "test1"}},
)

Parent run 9224183c-9537-4805-b63c-2f9ca401792c not found for run 5a6813bc-5d54-4073-b2b0-4a80548fe686. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_RtKlm0aQkorTIlnT8Zpgu8NL", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}

# base chain

In [28]:
model = ChatOpenAI()

base_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 뛰어나고 유능한 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        ("human", "{input}"),

    ]
)

base_chain = base_prompt | model

In [32]:
response = base_chain.invoke({"input": "안녕 나는 젖수라고 해"})
response.content

'안녕하세요, 젖수님! 어떤 도움이 필요하신가요? 부담 갖지 마시고 무엇이든 물어보세요.'

: 